MovieLens 는 recommender system 용 reference data 로 자주 이용됩니다. 이 데이터를 GroupLens 로부터 다운받고, 그 중 (user, item) rating 행렬을 로딩하는 함수를 `soydata.data.external` 에 추가하였습니다. 이 데이터의 저작권은 모두 GroupLens 에 있으니 해당 데이터를 이용하여 공개용 실험을 수행하실 때에는 반드시 아래의 링크의 README 를 읽어주세요.

In [1]:
import soydata
from soydata.data.external.movielens import load_rating

print(f'soydata=={soydata.__version__}\n')

user_item, _ = load_rating(size='20m')
print(user_item.shape)

soydata==0.1.0

This function downloads MovieLens data from GroupLens
Please read first http://files.grouplens.org/datasets/movielens/ml-20m-README.html
All permissions are in GroupLens, and this function is an external utility to conventiently use MovieLens data.

(138494, 131263)


`user_item` 행렬의 크기를 측정합니다. empty item, empty user 가 존재합니다.

In [2]:
import numpy as np

user_indices, item_indices = user_item.nonzero()
user_size = np.bincount(user_indices, minlength=user_item.shape[0])
item_size = np.bincount(item_indices, minlength=user_item.shape[1])

n_empty_user = np.where(user_size == 0)[0].shape[0]
n_empty_item = np.where(item_size == 0)[0].shape[0]

print(f'n empty user = {n_empty_user}')
print(f'n empty item = {n_empty_item}')

n empty user = 1
n empty item = 104519


Collaborative Filtering 은 한 사용자 u 의 representation 이 주어졌을 때 이와 비슷한 representation 을 지니는 다른 사용자들을 찾은 뒤, 그 사용자들이 보았지만, u 가 보지 않은 아이템을 추천하는 방법입니다. 가장 간단하게는 (user, item) 행렬을 user representation 으로 이용하고, 한 사용자 u 와 가까운 다른 사용자를 탐색하기 위하여 모든 사용자들과의 거리를 계산합니다.

이러한 인터페이스는 재활용 가능하니 Similar rows 를 탐색하는 간단한 Finder 를 만듭니다. Query representation 이 주어질 때와 user index 가 주어질 때를 모두 가정하여 `get_similars()` 와 `get_similars_from_vector()` 를 만듭니다. 각 함수의 구성은 `appendix_collaborative_filtering_codebook.ipynb` 를 참고하세요.

In [3]:
from sklearn.metrics import pairwise_distances


class SimpleSimilarFinder:
    def __init__(self, matrix, metric='cosine'):
        self.matrix = matrix
        self.metric = metric
        self.n_rows, self.n_features = matrix.shape

    def get_similars(self, idx, topk=10):
        if not (0 <= idx < self.n_rows):
            raise ValueError(f'Unknown query: {idx}')
        query_vec = self.matrix[idx,:]
        return self.get_similars_from_vector(query_vec, topk)

    def get_similars_from_vector(self, query, topk=10):
        if len(query.shape) == 1:
            query = query.reshape(1,-1)

        dist = pairwise_distances(query, self.matrix, metric=self.metric)
        similar_idx = dist.argsort(axis=1)[:,:topk]

        cols = similar_idx.flatten()
        rows = np.repeat(np.arange(query.shape[0]), repeats=topk)
        similar_dist = dist[rows, cols].reshape(-1,topk)

        return similar_idx, similar_dist

Item rating vectors 를 user representation 으로 이용합니다. user 1 과 item rating vectors 가 비슷한 사용자들 10 명을 탐색해 봅니다. `candidates_history` 는 그 사용자들의 item rating vectors 입니다.

In [4]:
user_index = SimpleSimilarFinder(user_item)
similar_users, similar_dist = user_index.get_similars(1)
similar_sim = 1 - similar_dist

print(similar_users.shape)
print(similar_dist.shape)
print(similar_sim.shape)

candidates_history = user_item[similar_users.flatten()]
print(candidates_history.shape)

(1, 10)
(1, 10)
(1, 10)
(10, 131263)


이를 이용하는 CF 엔진을 만듭니다. 이 엔진에는 `recommender()` 라는 함수가 있습니다. 이는 사용자 아이디가 주어지면 비슷한 다른 사용자를 탐색하고, 그 사용자들 간의 유사도 `similar_sim` 을 반영하여 영화별 평점들 모두 더합니다. 그리고 해당 사용자 u 가 이미 본 아이템에 대해서는 점수를 0 으로 만들어 평점을 부여하지 않은 컨텐츠만 추천이 되도록 만듭니다. 이 과정을 통하여 추천할 후보군을 생성합니다. 때로는 특정한 규칙들을 이용하여 re-ranking 을 수행할 수도 있습니다. 이는 `TODO` 로 남겨뒀으니 자유롭게 구현해 보시기 바랍니다. 최종적으로 `topk_items` 를 출력합니다.

In [5]:
from time import time


class CollaborativeFiltering:
    def __init__(self, user_index, user_history):
        self.user_index = user_index
        self.user_history = user_history
        self.n_users = user_history.shape[0]
        self.popular_by_scores, self.popular_by_view = prepare_most_popular(user_history)

    def recommender(self, user_idx, topk_users=10, topk_items=3):
        t = time()
        _, already_seen = self.user_history[user_idx,:].nonzero()

        similar_users, similar_dist = self.user_index.get_similars(user_idx, topk_users)
        similar_sim = 1 - similar_dist
        candidates_history = self.user_history[similar_users.flatten()]

        # with user rating
        # shape = (1, num items)
        candidate_scores = np.dot(similar_sim, candidates_history.todense())
        # casting numpy.matrix -> numpy.ndarray
        candidate_scores = np.array(candidate_scores).reshape(-1)

        # remove already seen items
        candidate_scores[already_seen] = 0
        # argsort and reverse
        candidate_indices = candidate_scores.argsort()[::-1]

        # remove score-zero items
        n_positive = np.where(candidate_scores > 0)[0].shape[0]
        recommended_indices = candidate_indices[:n_positive]

        # re-ranking
        # TODO

        # select top items
        if topk_items > 0:
            recommended_indices = recommended_indices[:topk_items]

        # with score
        recommended_scores = candidate_scores[recommended_indices]

        # check processing time
        t = time() - t
        n_reco = recommended_scores.shape[0]
        print(f'recommend {n_reco} items from {n_positive} items with {t:.4} sec.')

        return recommended_indices, recommended_scores

    def most_popular(self, topk, by='score'):
        if by == 'score':
            self.popular_by_scores[:topk]
        return self.popular_by_view[:topk]

def prepare_most_popular(user_history):
    score_sum = user_history.sum(axis=0).reshape(-1)
    view_count = np.bincount(
        user_history.nonzero()[1],
        minlength = user_history.shape[1]
    )
    popular_by_scores = score_sum.argsort()[::-1]
    popular_by_view = view_count.argsort()[::-1]
    return popular_by_scores, popular_by_view

직접 만든 CF 를 이용하여 1 번 사용자에 대해 10 개의 아이템을 추천해봅니다. 약 2.3 초의 계산시간이 걸렸습니다. 이는 (user, item) 행렬의 크기가 크고, 모든 사용자간의 거리를 계산했기 때문입니다.

In [6]:
cf = CollaborativeFiltering(user_index, user_item)
items, scores = cf.recommender(1, topk_users=5, topk_items=10)

recommend 10 items from 494 items with 2.309 sec.


이번에는 이전에 공부한 TruncatedSVD 를 이용하여 (user, item) 행렬의 features 를 100 개로 줄여봅니다.

In [7]:
%%time

# with SVD
from sklearn.decomposition import TruncatedSVD

user_svd = TruncatedSVD(n_components=100).fit_transform(user_item)
user_svd.shape

CPU times: user 1min 3s, sys: 36.3 s, total: 1min 39s
Wall time: 41.8 s


이는 user representation 이므로 앞서 만든 `SimpleSimilarFinder` 에 입력할 수 있습니다. 이를 이용하여 동일한 작업을 수행하니 시간이 0.34 초 수준으로 줄어들었습니다. 더하여, 앞서서는 494 개의 추천 후보가 만들어졌지만, 이번에는 602 개의 추천 후보가 만들어졌습니다. 이는 sparse vector 를 이용할 때에는 A 라는 영화를 본 사람들의 대부분이 B 라는 영화를 본다 하여도 1 번 사용자가 B 를 보지 않았다면, B 를 본 다른 사용자를 비슷하다고 판단하지 않지만, SVD 에 의하여 A 와 B 영화가 하나의 principal components 로 묶였기 때문에 비슷한 사용자의 폭이 훨씬 늘어나기 때문입니다.

In [8]:
user_index_svd = SimpleSimilarFinder(user_svd)
cf_svd = CollaborativeFiltering(user_index_svd, user_item)
items, scores = cf_svd.recommender(1, topk_users=5, topk_items=10)

recommend 10 items from 602 items with 0.3473 sec.


하지만 여전히 0.34 초는 느립니다. 이번에는 비슷한 사용자를 탐색하기 위해 BallTree 를 이용해봅니다. BallTree 는 cosine distance 를 지원하지 않습니다. 하지만 L2 norm 이 1 인 unit vector 는 cosine distance 로 가까운 벡터 순서와 euclidean distance 로 가까운 벡터 순사가 같습니다. 그러므로 svd 의 학습 결과를 L2 normalizing 한 뒤, BallTree 에 입력합니다.

In [9]:
from sklearn.preprocessing import normalize
from sklearn.neighbors import BallTree

user_svd_norm = normalize(user_svd)
# ball_tree = BallTree(user_svd, metric='cosine')
ball_tree = BallTree(user_svd_norm, metric='euclidean')

1 번 사용자와 비슷한 10 명의 사용자를 탐색하는 시간이 0.038 초로 많이 줄었습니다.

In [10]:
%%time
similar_euc, similar_idxs = ball_tree.query(user_svd_norm[1].reshape(1,-1), k=10, return_distance=True)

CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 38.6 ms


Unit vector 에서의 euclidean distance 와 cosine distance 를 변경한 뒤,

In [11]:
similar_cos = (similar_euc ** 2) / 2

print(similar_idxs)
print(similar_euc)
print(similar_cos)

[[     1 110069  13605   5366  47783  62235   7412  50679 106779  81275]]
[[0.         0.65237934 0.6564257  0.66224347 0.6805567  0.69439393
  0.69458269 0.69896183 0.70051947 0.70168124]]
[[0.         0.2127994  0.21544735 0.21928321 0.23157871 0.24109147
  0.24122256 0.24427382 0.24536377 0.24617828]]


Euclidean distance 로부터 유도된 Cosine distance 의 값과 같은지 확인해 봅니다.

In [12]:
from sklearn.metrics import pairwise_distances

dist = pairwise_distances(user_svd_norm[1].reshape(1,-1), user_svd, metric='cosine').reshape(-1)
idxs = dist.argsort()[:10]
dist = dist[idxs]

print(idxs)
print(dist)

[     1 110069  13605   5366  47783  62235   7412  50679 106779  81275]
[0.         0.2127994  0.21544735 0.21928321 0.23157871 0.24109147
 0.24122256 0.24427382 0.24536377 0.24617828]


BallTree 를 이용하여 Cosine distance 기준으로 비슷한 사용자를 찾을 수 있음을 확인했으니 BallTree 를 기반으로 새로운 SimilarFinder 를 만듭니다. 함수는 위와 동일합니다.

In [13]:
from sklearn.neighbors import BallTree


class BallTreeSimilarFinder:
    def __init__(self, matrix, leaf_size=40, **kargs):
        self.matrix = matrix
        self.index = BallTree(matrix, leaf_size, 'euclidean', **kargs)
        self.n_rows, self.n_features = matrix.shape

    def get_similars(self, idx, topk=10):
        if not (0 <= idx < self.n_rows):
            raise ValueError(f'Unknown query: {idx}')
        query_vec = self.matrix[idx,:]
        return self.get_similars_from_vector(query_vec, topk)

    def get_similars_from_vector(self, query, topk=10):
        if len(query.shape) == 1:
            query = query.reshape(1,-1)

        similar_euc, similar_idx = self.index.query(query, topk, return_distance=True)
        similar_dist = (similar_euc ** 2) / 2
        return similar_idx, similar_dist

새로 만든 indexer 를 이용하여 동일한 추천 작업을 수행해 봅니다. 0.34 초에서 0.043 초로 시간이 많이 단축되었습니다.

In [14]:
user_index_bt = BallTreeSimilarFinder(user_svd_norm)
cf_bt = CollaborativeFiltering(user_index_bt, user_item)
items, scores = cf_bt.recommender(1, topk_users=5, topk_items=10)

recommend 10 items from 602 items with 0.04343 sec.
